In [1]:
from dotenv import load_dotenv
load_dotenv("../.env")

True

In [2]:
import pandas as pd
from pydantic.v1 import BaseModel, Field
from langchain_core.prompts import ChatPromptTemplate
from langchain_aws import ChatBedrock
from langchain_community.cache import SQLiteCache
from langchain.globals import set_llm_cache
import json

In [3]:
set_llm_cache(SQLiteCache(database_path="./temp_folder/llm_cache.sqlite.db"))

In [4]:
from rapidfuzz import fuzz

def find_matching_values(value, column, df):
    uniques = pd.Series(df[column].unique())
    uniques_scores = uniques.apply(lambda x: fuzz.partial_ratio(x.lower(), value.lower()))
    uniques_scores.index = uniques
    return uniques_scores.sort_values(ascending=False).head(5).index.to_list()

In [5]:
class Column(BaseModel):
    name: str = Field(description='name for the column')
    data_type: str = Field(description='python data type of the column')
    primary_key: bool = Field(description='whether the column can be a primary key')
    foreign_key: bool = Field(description='whether the column can be a foreign key')
    unique: bool = Field(description='whether the column is unique') ###
    nullable: bool = Field(description='whether the column is nullable')
    is_numeric: bool = Field(description='whether the column is numeric')
    is_categorical: bool = Field(description='whether the column is categorical')
    is_datetime: bool = Field(description='whether the column is a datetime')
    datetime_format: str| None = Field(description='format of the datetime in  strftime format')
    description: str = Field(description='description of the column, add all the unique values to this description also') # unique values in description


class TableResponse(BaseModel):
    name: str = Field(description='appropriate name for the table')
    description: str = Field(description='description of the table')
    columns: list[Column]

def get_df_metadata(df: pd.DataFrame):
    system_prompt = """
You will be given csv sample of data. Understand the data and give metadata about thable and columns in the format specified


return json with the structure provided"""

    user_prompt = """
    Table Sample CSV String
    >>>
    {table_csv_string}
    <<<

    Output of `df.describe()`
    >>>
    {describe_output}
    <<<

    Output of `df.nunique()`
    >>>
    {nunique_output}
    <<<
    
    Unique values for each column
    >>>
    {unique_column_values}

    <<<
    
    """
    unique_values = {}

    for col in df.columns:
        unique_vals = df[col].unique()
        if len(unique_vals) < 10:
            unique_values[col] = unique_vals.tolist()
        else:
            unique_values[col] = f"{unique_vals.tolist()[:10]} there are more values here"

        prompt_template = ChatPromptTemplate.from_messages(
            [
                ("system", system_prompt),
                ("human", user_prompt),
            ]
        )

    chain = (
        prompt_template
        | ChatBedrock(
            model_id="anthropic.claude-3-5-sonnet-20240620-v1:0",
            model_kwargs=dict(temperature=0, max_tokens = 8000),

        ).with_structured_output(TableResponse)
    )


    return chain.invoke({"table_csv_string": df.sample(10, random_state=0).to_csv(index=False), "describe_output": df.describe().to_csv(), "nunique_output": df.nunique().to_csv(header=False),"unique_column_values":unique_values})

In [6]:
def process_datetime_columns(df, metadata: TableResponse):
    column_metadata_df = pd.DataFrame(metadata.dict()['columns'])
    df = df.copy()
    for _, row in column_metadata_df.iterrows():
        if row['is_datetime']:
            df[row['name']] = pd.to_datetime(df[row['name']], format=row['datetime_format'])
    return df

In [13]:
# df = pd.read_csv('./data/stylumia_all_data.csv')
# df = pd.read_parquet("data/processed_data/outdoor_lighting_products.parquet")
df = pd.read_parquet("data/processed_data/zip_code_data.parquet")


In [14]:
df.head()

,Retailer,Product Name,Product URL,Brand,Rank,Top Seller,Power Source,Type,Color Changing,brightness (lumens),brightness (lumens) Bucket,in_store_availablity_48162,in_store_availablity_63111,in_store_availablity_49418,Availabe_State_Michigan,Availabe_State_Missouri,Availabe_Region_Rural,Availabe_Region_Urban,Availabe_Region_Coastal
0,B,"Enchanted Garden 6.3"" Glass Flameless Candle L...",https://www.menards.com/main/outdoors/outdoor-...,Enchanted Garden,Poor Seller,None,battery powered,Lanterns,False,NaN,None,False,True,True,True,True,False,True,True
1,B,"Enchanted Garden 14"" Metal & Glass Solar LED L...",https://www.menards.com/main/outdoors/outdoor-...,Enchanted Garden,Good Seller,Yes,solar powered,Lanterns,False,NaN,None,True,True,True,True,True,True,True,True
2,B,"Enchanted Garden 19.5"" Blue Ceramic Solar Lantern",https://www.menards.com/main/outdoors/outdoor-...,Enchanted Garden,Good Seller,Yes,solar powered,Lanterns,False,NaN,None,True,True,True,True,True,True,True,True
3,B,"Enchanted Garden 10-5/8"" Metal Solar Green Han...",https://www.menards.com/main/outdoors/outdoor-...,Enchanted Garden,Poor Seller,None,solar powered,Lanterns,False,NaN,None,False,False,True,True,False,False,False,True
4,B,"Enchanted Garden 12"" Bamboo Solar LED Lantern ...",https://www.menards.com/main/outdoors/outdoor-...,Enchanted Garden,Good Seller,Yes,solar powered,Lanterns,False,NaN,None,True,True,True,True,True,True,True,True


In [15]:
# df["Retailer"].unique()

In [16]:
# retailer_psudo_name_dict = {"Lowe's":"A" , 'Menards':"B" , 'Home Depot':"C" , 'Walmart':"D"}

In [17]:
# df["Retailer"] = df["Retailer"].map(retailer_psudo_name_dict)
# df.head()

In [38]:
df["Retailer"].unique()

array(['B', 'C', 'D', 'A'], dtype=object)

In [9]:
# # generate metadata for every 30 columns, always include the first 5 columns
# metadatas = []
# for i in range(0, len(df.columns), 30):
#     print(f"{i} to {i+30}")
#     df_column_sample = df.iloc[:, i:i+30]
#     if i > 0:
#         df_column_sample = pd.concat([df.iloc[:, :5], df_column_sample], axis=1)
#     metadata = get_df_metadata(df_column_sample)
#     metadatas.append(metadata)

In [18]:
metadata = get_df_metadata(df)

In [19]:
metadata

TableResponse(name='Outdoor_Lighting_Products', description='This table contains information about various outdoor lighting products from different retailers, including product details, specifications, and availability.', columns=[Column(name='Retailer', data_type='string', primary_key=False, foreign_key=False, unique=False, nullable=False, is_numeric=False, is_categorical=True, is_datetime=False, datetime_format=None, description='The retailer selling the product. Unique values: A, B, C, D'), Column(name='Product Name', data_type='string', primary_key=False, foreign_key=False, unique=True, nullable=False, is_numeric=False, is_categorical=False, is_datetime=False, datetime_format=None, description='The name of the outdoor lighting product'), Column(name='Product URL', data_type='string', primary_key=True, foreign_key=False, unique=True, nullable=False, is_numeric=False, is_categorical=False, is_datetime=False, datetime_format=None, description='The URL of the product page'), Column(nam

In [20]:
pd.DataFrame(metadata.dict()['columns'])

,name,data_type,primary_key,foreign_key,unique,nullable,is_numeric,is_categorical,is_datetime,datetime_format,description
0,Retailer,string,False,False,False,False,False,True,False,None,The retailer selling the product. Unique value...
1,Product Name,string,False,False,True,False,False,False,False,None,The name of the outdoor lighting product
2,Product URL,string,True,False,True,False,False,False,False,None,The URL of the product page
3,Brand,string,False,False,False,False,False,True,False,None,The brand of the product. Examples: Enchanted ...
4,Rank,string,False,False,False,True,False,True,False,None,The sales rank of the product. Values: Poor Se...
5,Top Seller,boolean,False,False,False,True,False,True,False,None,Indicates if the product is a top seller. Valu...
6,Power Source,string,False,False,False,True,False,True,False,None,The power source of the product. Values: batte...
7,Type,string,False,False,False,True,False,True,False,None,The type of outdoor lighting product. Examples...
8,Color Changing,boolean,False,False,False,False,False,True,False,None,Indicates if the product has color-changing ca...
9,brightness (lumens),float,False,False,False,True,True,False,False,None,The brightness of the product in lumens. Range...


In [21]:
metadata.name

'Outdoor_Lighting_Products'

In [22]:
metadata.description

'This table contains information about various outdoor lighting products from different retailers, including product details, specifications, and availability.'

In [23]:
df = process_datetime_columns(df, metadata)

In [24]:
df.head()

,Retailer,Product Name,Product URL,Brand,Rank,Top Seller,Power Source,Type,Color Changing,brightness (lumens),brightness (lumens) Bucket,in_store_availablity_48162,in_store_availablity_63111,in_store_availablity_49418,Availabe_State_Michigan,Availabe_State_Missouri,Availabe_Region_Rural,Availabe_Region_Urban,Availabe_Region_Coastal
0,B,"Enchanted Garden 6.3"" Glass Flameless Candle L...",https://www.menards.com/main/outdoors/outdoor-...,Enchanted Garden,Poor Seller,None,battery powered,Lanterns,False,NaN,None,False,True,True,True,True,False,True,True
1,B,"Enchanted Garden 14"" Metal & Glass Solar LED L...",https://www.menards.com/main/outdoors/outdoor-...,Enchanted Garden,Good Seller,Yes,solar powered,Lanterns,False,NaN,None,True,True,True,True,True,True,True,True
2,B,"Enchanted Garden 19.5"" Blue Ceramic Solar Lantern",https://www.menards.com/main/outdoors/outdoor-...,Enchanted Garden,Good Seller,Yes,solar powered,Lanterns,False,NaN,None,True,True,True,True,True,True,True,True
3,B,"Enchanted Garden 10-5/8"" Metal Solar Green Han...",https://www.menards.com/main/outdoors/outdoor-...,Enchanted Garden,Poor Seller,None,solar powered,Lanterns,False,NaN,None,False,False,True,True,False,False,False,True
4,B,"Enchanted Garden 12"" Bamboo Solar LED Lantern ...",https://www.menards.com/main/outdoors/outdoor-...,Enchanted Garden,Good Seller,Yes,solar powered,Lanterns,False,NaN,None,True,True,True,True,True,True,True,True


In [25]:
file_name = "outdoor_lighting_products_renamed_zipcode"

In [26]:

df.to_parquet(f"./data/processed_data/{file_name}.parquet", index=False)
with open(f"./data/processed_data/{file_name}_metadata.json", "w") as f:
    f.write(json.dumps(metadata.dict(), indent=4))

In [27]:
df = pd.read_parquet(f"./data/processed_data/{file_name}.parquet")

In [28]:
df.dtypes

Retailer                       object
Product Name                   object
Product URL                    object
Brand                          object
Rank                           object
Top Seller                     object
Power Source                   object
Type                           object
Color Changing                   bool
brightness (lumens)           float64
brightness (lumens) Bucket     object
in_store_availablity_48162       bool
in_store_availablity_63111       bool
in_store_availablity_49418       bool
Availabe_State_Michigan          bool
Availabe_State_Missouri          bool
Availabe_Region_Rural            bool
Availabe_Region_Urban            bool
Availabe_Region_Coastal          bool
dtype: object

In [29]:
df.head()

,Retailer,Product Name,Product URL,Brand,Rank,Top Seller,Power Source,Type,Color Changing,brightness (lumens),brightness (lumens) Bucket,in_store_availablity_48162,in_store_availablity_63111,in_store_availablity_49418,Availabe_State_Michigan,Availabe_State_Missouri,Availabe_Region_Rural,Availabe_Region_Urban,Availabe_Region_Coastal
0,B,"Enchanted Garden 6.3"" Glass Flameless Candle L...",https://www.menards.com/main/outdoors/outdoor-...,Enchanted Garden,Poor Seller,None,battery powered,Lanterns,False,NaN,None,False,True,True,True,True,False,True,True
1,B,"Enchanted Garden 14"" Metal & Glass Solar LED L...",https://www.menards.com/main/outdoors/outdoor-...,Enchanted Garden,Good Seller,Yes,solar powered,Lanterns,False,NaN,None,True,True,True,True,True,True,True,True
2,B,"Enchanted Garden 19.5"" Blue Ceramic Solar Lantern",https://www.menards.com/main/outdoors/outdoor-...,Enchanted Garden,Good Seller,Yes,solar powered,Lanterns,False,NaN,None,True,True,True,True,True,True,True,True
3,B,"Enchanted Garden 10-5/8"" Metal Solar Green Han...",https://www.menards.com/main/outdoors/outdoor-...,Enchanted Garden,Poor Seller,None,solar powered,Lanterns,False,NaN,None,False,False,True,True,False,False,False,True
4,B,"Enchanted Garden 12"" Bamboo Solar LED Lantern ...",https://www.menards.com/main/outdoors/outdoor-...,Enchanted Garden,Good Seller,Yes,solar powered,Lanterns,False,NaN,None,True,True,True,True,True,True,True,True
